【ハイパーパラメタ・チューニング】<br>
これまで機械学習の手法として、SVMやランダムフォレストを使って、分類や予測をやってきましたが、<br>
実は、SVMやランダムフォレストには、ハイパーパラメータという「手動」で設定する必要がある変数があります。<br>
前にやった時は、ハイパーパラメータをデフォルト値のまま実行したことになります。<br>
今回は、また再びタイタニックを使い、このハイパーパラメータを見ていきましょう。

問1<br>
機械学習アルゴリズムのハイパーパラメータがどういう役割か、<br>また実際にSVM・ランダムフォレストの主なハイパーパラメータにどういうものがあるか、<br>またデフォルトの値が何か、調べてください。

<br>
ハイパーパラメータとは、機械学習の実行による推論や予測では決定されない、<br>
プログラム実施者が任意に決定するパラメータを言う。なお、「パラメータ」<br>
は、機械学習モデルが学習の過程で自動的に最適化を行うものを指す。
<br>
<主なハイパーパラメータ><br>
1. SVM<br>
① kernel(デフォルト:rbf - radial basis function)<br>
カーネルは、SVMの演算を行う'関数'、他に、'linear','poly'等の<br>
選択肢がある<br>
② C(コスト、デフォルト:1.0 <-実数)<br>
SVMの損失関数の誤差の係数(誤分類を許容する程度)<br>
③ gamma(γ、デフォルト:1/(特徴量の数*分散) <- 実数)<br>
kernel が 'rbf'・'poly'・'sigmoid'の際のカーネル関数の係数<br>
2. ランダムフォレスト<br>
① n_estimators(デフォルト:100-整数)<br>
ランダムフォレストは、「決定木」に設定した条件で元の集合を2分割し<br>
さらに別れた先で再度2分割する、を繰り返すが、その決定木の数<br>
② max_depth(デフォルト:None)<br>
決定木のノードの深さ(=階層)<br>
③ max_features(デフォルト:特徴量nの平方根)<br>
決定木それぞれに対し、ランダムに指定する特徴量の数。

問2<br>
① タイタニックの'train.csv'を読み込み、予測精度に影響が少ない特徴量を外し、<br>
データを学習用:テスト用=7:3に分割し、SVMの学習モデルを作成し、予測精度を求めてください。<br>
(C,gammaの値はデフォルトで)

In [13]:
import pandas as pd

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
cd /content/drive/MyDrive/AIStudy/git/exc_hyperpara_titanic

/content/drive/MyDrive/AIStudy/git/exc_hyperpara_titanic


In [16]:
df = pd.read_csv('./data/train.csv')

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [18]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [19]:
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [23]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [21]:
x = df.drop(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp','Parch','Ticket', 'Cabin'], axis=1)
y = df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3,random_state=7)

In [22]:
clf = SVC()
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
acc = accuracy_score(pred, y_test)
print('accuracy score : {:.5f}' .format(acc))

accuracy score : 0.65299
